
# EU-DEMO 2017 reference breakdown and equilibrium benchmark

with
- 300mw out
- 2 hr flat top

In [ ]:
import json
import os
import sys
from copy import deepcopy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython import get_ipython

from bluemira.base.constants import raw_uc
from bluemira.base.file import get_bluemira_path
from bluemira.base.look_and_feel import bluemira_print
from bluemira.display import plot_2d, plot_defaults
from bluemira.equilibria.coils import Coil, CoilSet
from bluemira.equilibria.equilibrium import Breakdown, Equilibrium
from bluemira.equilibria.grid import Grid
from bluemira.equilibria.opt_constraints import (
    CoilFieldConstraints,
    CoilForceConstraints,
    FieldNullConstraint,
    IsofluxConstraint,
    MagneticConstraintSet,
    PsiBoundaryConstraint,
)
from bluemira.equilibria.opt_problems import (
    BreakdownCOP,
    MinimalCurrentCOP,
    OutboardBreakdownZoneStrategy,
    UnconstrainedTikhonovCurrentGradientCOP,
)
from bluemira.equilibria.physics import calc_psib
from bluemira.equilibria.profiles import CustomProfile
from bluemira.equilibria.run import PulsedNestedPositionCOP
from bluemira.equilibria.solve import PicardIterator
from bluemira.geometry.face import BluemiraFace
from bluemira.geometry.plane import BluemiraPlane
from bluemira.geometry.tools import make_polygon, offset_wire, slice_shape, split_wire
from bluemira.utilities.optimiser import Optimiser
from bluemira.utilities.positioning import PathInterpolator, PositionMapper
from eudemo.pf_coils.tools import make_coil_mapper, make_pf_coil_path


Load the reference equilibria from EFDA_D_2MUW9R

In [ ]:
plot_defaults()

try:
    get_ipython().run_line_magic("matplotlib", "qt")
except AttributeError:
    pass

path = get_bluemira_path("equilibria", subfolder="examples")
name = "EUDEMO_2017_CREATE_SOF_separatrix.json"
filename = os.sep.join([path, name])
with open(filename, "r") as file:
    data = json.load(file)

sof_xbdry = data["xbdry"]
sof_zbdry = data["zbdry"]


Import keep out zones

In [ ]:
xl_dict = pd.read_excel(sys.argv[1], None)

koz_LP_k = koz_UP_k = TF_inner_k = TF_outer_k = None
for key in xl_dict.keys():
    key_low = key.lower()
    if "keep" in key_low:
        if "lp" in key_low:
            koz_LP_k = key
        elif "up" in key_low:
            koz_UP_k = key
    if "tf" in key_low:
        if "inner" in key_low:
            TF_inner_k = key
        elif "outer" in key_low:
            TF_outer_k = key


def get_unit(column_name):
    try:
        return column_name.split("(")[1].strip(") ")
    except IndexError:
        return "mm"


# Asummes the units are the same for both columns
koz_LP_raw = xl_dict[koz_LP_k]
koz_UP_raw = xl_dict[koz_UP_k]

TF_inner_raw = xl_dict[TF_inner_k]
TF_outer_raw = xl_dict[TF_outer_k]

koz_LP = raw_uc(
    koz_LP_raw.to_numpy()[:, :2].astype(float), get_unit(koz_LP_raw.columns[0]), "m"
).T

koz_UP = raw_uc(
    koz_UP_raw.to_numpy()[:, :2].astype(float), get_unit(koz_UP_raw.columns[0]), "m"
).T

TF_inner = raw_uc(
    TF_inner_raw.to_numpy()[:, :2].astype(float), get_unit(TF_inner_raw.columns[0]), "m"
).T

TF_outer = raw_uc(
    TF_outer_raw.to_numpy()[:, :2].astype(float), get_unit(TF_outer_raw.columns[0]), "m"
).T


Make the same CoilSet as CREATE

In [ ]:
x = [5.4, 14, 17.75, 17.75, 14.0, 7, 2.77, 2.77, 2.77, 2.77, 2.77]
z = [9.26, 7.9, 2.5, -2.5, -7.9, -10.5, 7.07, 4.08, -0.4, -4.88, -7.86]
dx = [0.6, 0.7, 0.5, 0.5, 0.7, 1.0, 0.4, 0.4, 0.4, 0.4, 0.4]
dz = [0.6, 0.7, 0.5, 0.5, 0.7, 1.0, 2.99 / 2, 2.99 / 2, 5.97 / 2, 2.99 / 2, 2.99 / 2]

# crude movement of coils
x[0] = min(x[0], min(koz_UP[0])) - dx[0]
x[1] = max(x[1], max(koz_UP[0])) + dx[1]
x[4] = x[4] + 2 * dx[4]
x[5] = min(x[5], min(koz_LP[0]))

z[1] = z[1] - dz[1]
z[4] = z[4] + 2 * dz[4]

# Matti's positions
# x[:6] = np.array([4, 14.54, 17.75, 17.75, 15.4, 7.0])
# z[:6] = np.array([9.26, 7.25, 2.5, -2.5, -6.55, -10.5])

# create coilset
coils = []
j = 1
for i, (xi, zi, dxi, dzi) in enumerate(zip(x, z, dx, dz)):
    if j > 6:
        j = 1
    ctype = "PF" if i < 6 else "CS"
    coil = Coil(
        xi,
        zi,
        current=0,
        dx=dxi,
        dz=dzi,
        ctype=ctype,
        name=f"{ctype}_{j}",
    )
    coils.append(coil)
    j += 1

coilset = CoilSet(*coils)

# Assign current density and peak field constraints
coilset.assign_material("CS", j_max=16.5e6, b_max=12.5)
coilset.assign_material("PF", j_max=12.5e6, b_max=11)
coilset.fix_sizes()
coilset.discretisation = 0.3


Plot keep out zones and coils

In [ ]:
# Adjust axis limits
fig, ax = plt.subplots()
axis_pad = 1
ax.set_xlim(0, max(np.max(koz_UP[0]), np.max(koz_LP[0])) + axis_pad)
ax.set_ylim(
    -(max(np.max(koz_UP[1]), -np.min(koz_LP[1])) + axis_pad),
    max(np.max(koz_UP[1]), np.max(koz_LP[1])) + axis_pad,
)

# add TF and koz to plot
ax.plot(*koz_UP)
ax.plot(*koz_LP)
ax.plot(*TF_inner)
ax.plot(*TF_outer)

coilset.plot(ax=ax)

In [ ]:
plt.draw()
plt.pause(0.001)


Define parameters

In [ ]:
# Machine parameters
I_p = 19.07e6  # A
beta_p = 1.141
l_i = 0.8
R_0 = 8.938
Z_0 = 0.027454
B_0 = 4.8901  # ???
A = 3.1
kappa_95 = 1.65
delta_95 = 0.33
tau_flattop = 2 * 3600  # 2hrs
v_burn = 4.220e-2  # V
c_ejima = 0.3

# Breakdown constraints (I can't quite get it with 3mT..) I've gotten close to 305 V.s,
# but only using a smaller low-field region.
# This is quite a sensitive optimisation, and is possibly a multi-modal space
# May want to think about optimising with a stochastic optimiser, and including
# a parametric location of the breakdown point...
x_zone = 9.84  # ??
z_zone = 0.0  # ??
r_zone = 2.0  # ??
b_zone_max = 0.003  # T

# Coil constraints
PF_Fz_max = 450
CS_Fz_sum = 300
CS_Fz_sep = 350

Use the same grid as CREATE (but less discretised):

In [ ]:
grid = Grid(2, 16.0, -9.0, 9.0, 100, 100)


Set up the Breakdown object

In [ ]:
field_constraints = CoilFieldConstraints(coilset, coilset.b_max, tolerance=1e-6)
force_constraints = CoilForceConstraints(
    coilset, PF_Fz_max, CS_Fz_sum, CS_Fz_sep, tolerance=1e-6
)

max_currents = coilset.get_max_current(0)


breakdown = Breakdown(deepcopy(coilset), grid)

bd_opt_problem = BreakdownCOP(
    breakdown.coilset,
    breakdown,
    OutboardBreakdownZoneStrategy(R_0, A, 0.225),
    optimiser=Optimiser("COBYLA", opt_conditions={"max_eval": 3000, "ftol_rel": 1e-6}),
    max_currents=max_currents,
    B_stray_max=1e-3,
    B_stray_con_tol=1e-6,
    n_B_stray_points=10,
    constraints=[
        field_constraints,
        force_constraints,
    ],
)

coilset = bd_opt_problem.optimise(x0=max_currents)
bluemira_print(f"Breakdown psi: {breakdown.breakdown_psi*2*np.pi:.2f} V.s")

# force breakdown flux to 320 Vs
breakdown_flux = 320  # breakdown.breakdown_psi*2*np.pi


Calculate SOF and EOF plasma boundary fluxes

In [ ]:
psi_sof = calc_psib(breakdown_flux, R_0, I_p, l_i, c_ejima)
psi_eof = psi_sof - tau_flattop * v_burn

# CREATE then knocked off an extra 10 V.s for misc plasma stuff I didnt look into

psi_sof -= 10
psi_eof -= 10


Set up a parameterised profile

In [ ]:
profiles = CustomProfile(
    np.array([86856, 86506, 84731, 80784, 74159, 64576, 52030, 36918, 20314, 4807, 0.0]),
    -np.array(
        [0.125, 0.124, 0.122, 0.116, 0.106, 0.093, 0.074, 0.053, 0.029, 0.007, 0.0]
    ),
    R_0=R_0,
    B_0=B_0,
    I_p=I_p,
)
# profile = BetaIpProfile(beta_p, I_p, R_0, B_0, shape=shape)

Solve the SOF and EOF equilibria

In [ ]:
reference_eq = Equilibrium(
    deepcopy(coilset),
    grid,
    profiles,
)

# Make a set of magnetic constraints for the equilibria... I got lazy here,
# this is just:
#   * LCFS boundary fluxes
#   * Field null at lower X-point
#   * divertor legs are not treated, but could easily be added

isoflux = IsofluxConstraint(
    np.array(sof_xbdry)[::10],
    np.array(sof_zbdry)[::10],
    sof_xbdry[0],
    sof_zbdry[0],
    tolerance=1e-3,
    constraint_value=0.25,  # Difficult to choose...
)
xp_idx = np.argmin(sof_zbdry)
x_point = FieldNullConstraint(
    sof_xbdry[xp_idx],
    sof_zbdry[xp_idx],
    tolerance=1e-3,
)

ref_opt_problem = UnconstrainedTikhonovCurrentGradientCOP(
    reference_eq.coilset,
    reference_eq,
    MagneticConstraintSet([isoflux, x_point]),
    gamma=1e-7,
)

program = PicardIterator(reference_eq, ref_opt_problem, fixed_coils=True, relaxation=0.2)
program()


sof_psi_boundary = PsiBoundaryConstraint(
    np.array(sof_xbdry)[::10],
    np.array(sof_zbdry)[::10],
    psi_sof / (2 * np.pi),
    tolerance=1.0,
)

optimiser = Optimiser("SLSQP", opt_conditions={"max_eval": 1000, "ftol_rel": 1e-6})
sof = deepcopy(reference_eq)

sof_opt_problem = MinimalCurrentCOP(
    sof.coilset,
    sof,
    optimiser=optimiser,
    max_currents=max_currents,
    constraints=[sof_psi_boundary, x_point],
)


eof_psi_boundary = PsiBoundaryConstraint(
    np.array(sof_xbdry)[::10],
    np.array(sof_zbdry)[::10],
    psi_eof / (2 * np.pi),
    tolerance=1.0,
)

eof = deepcopy(reference_eq)
eof_opt_problem = MinimalCurrentCOP(
    eof.coilset,
    eof,
    optimiser=optimiser,
    max_currents=max_currents,
    constraints=[eof_psi_boundary, x_point],
)

In [ ]:
# We'll store these so that we can look at them again later
old_coilset = deepcopy(coilset)
old_eq = deepcopy(reference_eq)

offset_val = np.max([dx, dz])
tf_outer = TF_outer[:, np.where(TF_outer[1] == np.max(TF_outer[1]))[0][0] :]
tf_outer = tf_outer[:, : np.where(tf_outer[0] == np.min(tf_outer[0]))[0][2]]
t_outer = np.array([tf_outer[0], np.zeros(len(tf_outer[0])), tf_outer[1]])

# full_path = offset_wire(make_polygon(t_outer), offset_val)

# # offset planes in x
# offset1 = min(koz_UP[0])
# offset2 = max(koz_UP[0])
# offset3 = min(koz_LP[0])

# p1 = BluemiraPlane.from_3_points([offset1, 0, 0], [offset1, 1, 0], [offset1, 1, 1])
# p2 = BluemiraPlane.from_3_points([offset2, 0, 0], [offset2, 1, 0], [offset2, 1, 1])
# p3 = BluemiraPlane.from_3_points([offset3, 0, 0], [offset3, 1, 0], [offset3, 1, 1])

# # offset plane in z
# offset4 = max(koz_LP[1])
# p4 = BluemiraPlane.from_3_points([0, 0, offset4], [0, 1, offset4], [1, 1, offset4])


# point1 = slice_shape(full_path, p1)[0]
# point2 = slice_shape(full_path, p2)[0]
# point3 = slice_shape(full_path, p3)[1]
# point4 = slice_shape(full_path, p4)[0]

# # create paths

# split1 = split_wire(full_path, point1)[0]
# split2_5 = split_wire(split_wire(full_path, point2)[1], point4)[0]
# split6 = split_wire(full_path, point3)[1]

# discr = split2_5.discretize(5).xyz
# split2, split3_5 = split_wire(split2_5, discr[:, 1], tolerance=1e-15)
# split3, split4_5 = split_wire(split3_5, discr[:, 2], tolerance=1e-15)
# split4, split5 = split_wire(split4_5, discr[:, 3], tolerance=1e-15)

# paths = [split1, split2, split3, split4, split5, split6]


face_koz_UP = BluemiraFace(
    make_polygon(np.array([koz_UP[0], np.zeros_like(koz_UP[0]), koz_UP[1]]))
)
face_koz_LP = BluemiraFace(
    make_polygon(np.array([koz_LP[0], np.zeros_like(koz_LP[0]), koz_LP[1]]))
)
position_mapper = make_coil_mapper(
    make_pf_coil_path(make_polygon(t_outer), offset_val / 2),
    [face_koz_UP, face_koz_LP],
    coilset.get_coiltype("PF")._coils,
)

In [ ]:
position_opt_problem = PulsedNestedPositionCOP(
    coilset,
    position_mapper,
    # sub_opt_problems=[sof_opt_problem, eof_opt_problem],
    sub_opt_problems=[eof_opt_problem],
    optimiser=Optimiser(
        "COBYLA", opt_conditions={"max_eval": 50, "ftol_rel": 1e-6, "xtol_rel": 1e-6}
    ),
    debug=False,
)

optimised_coilset = position_opt_problem.optimise(verbose=True)

In [ ]:
sof_pf_currents = sof.coilset.get_coiltype("PF").get_control_coils().current
eof_pf_currents = eof.coilset.get_coiltype("PF").get_control_coils().current
max_pf_currents = np.max(np.abs([sof_pf_currents, eof_pf_currents]), axis=0)

pf_coil_names = optimised_coilset.get_coiltype("PF").name

max_cs_currents = optimised_coilset.get_coiltype("CS").get_max_current()

max_currents = np.concatenate([max_pf_currents, max_cs_currents])

# for problem in [sof_opt_problem, eof_opt_problem]:
for problem in [eof_opt_problem]:
    for pf_name, max_current in zip(pf_coil_names, max_pf_currents):
        problem.eq.coilset[pf_name].resize(max_current)
        problem.eq.coilset[pf_name].fix_size()
        problem.eq.coilset[pf_name].discretisation = 0.3
    problem.set_current_bounds(max_currents)

In [ ]:
iterator = PicardIterator(
    sof, sof_opt_problem, plot=True, fixed_coils=True, relaxation=0.2
)
iterator()
iterator = PicardIterator(
    eof, eof_opt_problem, plot=True, relaxation=0.2, fixed_coils=True
)
iterator()

Plot the results

In [ ]:
f, ax = plt.subplots(1, 2)
ax1 = None
ax2 = ax[0]
ax3 = ax[1]

# breakdown.plot(ax1)
# breakdown.coilset.plot(ax1)
sof.plot(ax2)
sof.coilset.plot(ax2, label=True)
eof.plot(ax3)
eof.coilset.plot(ax3, label=True)

sof_psi = 2 * np.pi * sof.psi(*sof._x_points[0][:2])
eof_psi = 2 * np.pi * eof.psi(*eof._x_points[0][:2])

# ax1.set_title(f"Breakdown {ax[0].get_title()}")
ax2.set_title("SOF $\\psi_{b}$ = " + f"{sof_psi:.2f} V.s")
ax3.set_title("EOF $\\psi_{b}$ = " + f"{eof_psi:.2f} V.s")


for i in range(len(ax)):
    ax[i].set_xlim(0, max(np.max(koz_UP[0]), np.max(koz_LP[0])) + axis_pad)
    ax[i].set_ylim(
        -(max(np.max(koz_UP[1]), -np.min(koz_LP[1])) + axis_pad),
        max(np.max(koz_UP[1]), np.max(koz_LP[1])) + axis_pad,
    )
    ax[i].plot(*koz_UP)
    ax[i].plot(*koz_LP)
    ax[i].plot(*TF_inner)
    ax[i].plot(*TF_outer)

plt.show()